First, I will perform text preprocessing on the bank reviews.

In [ ]:
import json
import bz2
import regex
from tqdm import tqdm
from scipy import sparse

In [ ]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [02:14, 1492.21it/s]


In [ ]:
responses[99]

{'city': 'г. Саратов',
 'rating_not_checked': False,
 'title': 'Карта ко вкладу',
 'num_comments': 0,
 'bank_license': 'лицензия № 880',
 'author': 'ronnichka',
 'bank_name': 'Югра',
 'datetime': '2015-06-03 20:56:57',
 'text': 'Здравствуйте! Хотела написать, что мне месяц не выдают карту ко вкладу, ссылаясь на "нам же их из Самары везут" (на секундочку 5 часов езды от нашего города). Но! Прочитала, что людям 3,5 месяцев не выдают карту, и поняла, что у меня все хорошо, пока что. И подарок мне дали, и кулер в отделении есть. Так что я, конечно, готова ждать. Правда хотелось бы не очень долго.',
 'rating_grade': 3}

In [ ]:
df = pd.json_normalize(responses)[['city', 'bank_name', 'author', 'datetime', 'rating_grade', 'title', 'text', 'bank_license', 'num_comments', 'rating_not_checked']]

In [ ]:
df.head()

,city,bank_name,author,datetime,rating_grade,title,text,bank_license,num_comments,rating_not_checked
0,г. Москва,Бинбанк,uhnov1,2015-06-08 12:50:54,NaN,Жалоба,Добрый день! Я не являюсь клиентом банка и пор...,лицензия № 2562,0,False
1,г. Новосибирск,Сбербанк России,Foryou,2015-06-08 11:09:57,NaN,Не могу пользоваться услугой Сбербанк он-лайн,Доброго дня! Являюсь держателем зарплатной кар...,лицензия № 1481,0,False
2,г. Москва,Бинбанк,Vladimir84,2015-06-05 20:14:28,NaN,Двойное списание за один товар.,Здравствуйте! Дублирую свое заявление от 03.0...,лицензия № 2562,1,False
3,г. Ставрополь,Сбербанк России,643609,2015-06-05 13:51:01,NaN,Меняют проценты комиссии не предупредив и не ...,Добрый день!! Я открыл расчетный счет в СберБа...,лицензия № 1481,2,False
4,г. Челябинск,ОТП Банк,anfisa-2003,2015-06-05 10:58:12,NaN,Верните денежные средства за страховку,"04.03.2015 г. взяла кредит в вашем банке, заяв...",лицензия № 2766,1,False


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153499 entries, 0 to 153498
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   city                138325 non-null  object 
 1   bank_name           153499 non-null  object 
 2   author              153479 non-null  object 
 3   datetime            153499 non-null  object 
 4   rating_grade        88658 non-null   float64
 5   title               153499 non-null  object 
 6   text                153499 non-null  object 
 7   bank_license        153498 non-null  object 
 8   num_comments        153499 non-null  int64  
 9   rating_not_checked  153499 non-null  bool   
dtypes: bool(1), float64(1), int64(1), object(7)
memory usage: 10.7+ MB


In [ ]:
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from string import punctuation

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words('russian'))
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_text(text):
    if isinstance(text, str):
        text = re.sub(r'\S*https?:\S*', '', text)
        text = re.sub(r'[^a-zA-Zа-яА-Я]', ' ', text)
        text = text.lower()
        text = text.translate(str.maketrans('', '', punctuation))
        tokens = word_tokenize(text)
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
        return ' '.join(tokens)
    else:
        return ''

In [ ]:
df['processed_text'] = df['text'][:10000].apply(preprocess_text)

In [ ]:
print(df['processed_text'].apply(type).value_counts())

processed_text
<class 'float'>    143499
<class 'str'>       10000
Name: count, dtype: int64


In [ ]:
df = df[df['processed_text'].apply(lambda x: isinstance(x, str))]

I will build several topic models for the document collection with different numbers of topics and identify those where specific banks are mentioned.

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim import similarities

In [ ]:
texts = [text.split() for text in df['processed_text']]

In [ ]:
dictionary = Dictionary(texts)

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
len(corpus)

10000

In [ ]:
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [ ]:
corpus_tfidf

In [ ]:
index = similarities.MatrixSimilarity(corpus_tfidf)
sims = index[corpus_tfidf]

In [ ]:
from gensim.models import lsimodel

In [ ]:
lsi = lsimodel.LsiModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=5)

In [ ]:
lsi.show_topics()

[(0,
  '0.118*"деньги" + 0.114*"карты" + 0.112*"карту" + 0.102*"кредит" + 0.099*"г" + 0.099*"банк" + 0.095*"заявление" + 0.094*"банка" + 0.091*"это" + 0.090*"руб"'),
 (1,
  '-0.310*"звонки" + -0.236*"номер" + -0.190*"данных" + -0.182*"персональных" + -0.180*"телефона" + -0.173*"поступают" + -0.166*"базы" + -0.155*"удалить" + -0.147*"обработку" + -0.143*"согласия"'),
 (2,
  '-0.270*"кредит" + -0.238*"кредита" + 0.188*"карты" + -0.177*"страхования" + 0.173*"карту" + -0.162*"страховки" + -0.146*"договор" + 0.145*"деньги" + -0.141*"договора" + 0.137*"банкомат"'),
 (3,
  '0.352*"вклад" + 0.321*"вклада" + -0.180*"руб" + -0.127*"рублей" + -0.110*"банкомат" + -0.108*"г" + -0.103*"заявление" + -0.100*"средства" + 0.100*"очереди" + -0.095*"сбербанка"'),
 (4,
  '-0.278*"вклада" + -0.177*"вклад" + -0.160*"г" + -0.156*"рф" + -0.145*"договора" + -0.143*"средств" + -0.125*"денежных" + -0.122*"средства" + -0.107*"денежные" + -0.106*"персональных"')]

In [ ]:
lsi = lsimodel.LsiModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=10)

In [ ]:
lsi.show_topics()

[(0,
  '0.118*"деньги" + 0.114*"карты" + 0.112*"карту" + 0.102*"кредит" + 0.099*"г" + 0.099*"банк" + 0.095*"заявление" + 0.094*"банка" + 0.091*"это" + 0.090*"руб"'),
 (1,
  '-0.310*"звонки" + -0.236*"номер" + -0.189*"данных" + -0.181*"персональных" + -0.180*"телефона" + -0.173*"поступают" + -0.166*"базы" + -0.155*"удалить" + -0.147*"обработку" + -0.143*"согласия"'),
 (2,
  '0.272*"кредит" + 0.242*"кредита" + -0.187*"карты" + 0.174*"страхования" + -0.170*"карту" + 0.162*"страховки" + 0.145*"договор" + -0.145*"деньги" + 0.141*"договора" + -0.136*"банкомат"'),
 (3,
  '0.354*"вклад" + 0.319*"вклада" + -0.176*"руб" + -0.127*"рублей" + -0.109*"банкомат" + -0.109*"г" + -0.102*"средства" + 0.100*"очереди" + -0.100*"заявление" + -0.094*"сбербанка"'),
 (4,
  '-0.278*"вклада" + -0.178*"вклад" + -0.157*"рф" + -0.155*"г" + -0.144*"договора" + -0.143*"средств" + -0.125*"денежных" + -0.120*"средства" + -0.106*"персональных" + -0.105*"денежные"'),
 (5,
  '0.180*"р" + 0.174*"обязательствам" + 0.148*"ст

In [ ]:
lsi = lsimodel.LsiModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=20)

In [ ]:
lsi.show_topics()

[(0,
  '0.118*"деньги" + 0.114*"карты" + 0.112*"карту" + 0.102*"кредит" + 0.099*"г" + 0.099*"банк" + 0.095*"заявление" + 0.094*"банка" + 0.091*"это" + 0.090*"руб"'),
 (1,
  '-0.310*"звонки" + -0.236*"номер" + -0.189*"данных" + -0.181*"персональных" + -0.180*"телефона" + -0.173*"поступают" + -0.166*"базы" + -0.156*"удалить" + -0.147*"обработку" + -0.143*"согласия"'),
 (2,
  '-0.270*"кредит" + -0.241*"кредита" + 0.186*"карты" + -0.176*"страхования" + 0.173*"карту" + -0.161*"страховки" + 0.146*"деньги" + -0.146*"договор" + -0.142*"договора" + 0.136*"банкомат"'),
 (3,
  '-0.355*"вклад" + -0.319*"вклада" + 0.179*"руб" + 0.126*"рублей" + 0.110*"г" + 0.109*"банкомат" + -0.102*"очереди" + 0.101*"заявление" + 0.098*"средства" + 0.096*"сбербанка"'),
 (4,
  '0.276*"вклада" + 0.179*"вклад" + 0.157*"рф" + 0.156*"г" + 0.144*"договора" + 0.143*"средств" + 0.125*"денежных" + 0.123*"средства" + 0.108*"денежные" + 0.106*"персональных"'),
 (5,
  '-0.175*"р" + -0.174*"обязательствам" + -0.148*"страховки" 

From these topics, I noticed that discussions around Sberbank often centered on personal banking and loans, whereas VTB was frequently associated with investments and trading. This highlights the different market focuses and customer interactions of these banks.